#Snowflake Detector

In [ ]:
#Bonus cell just for executing linux commands





#Setup

In [ ]:
#Install dependecies
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 616 kB 15.0 MB/s 
     |████████████████████████████████| 1.1 MB 20.2 MB/s 
     |████████████████████████████████| 596 kB 51.4 MB/s 
     |████████████████████████████████| 1.2 MB 49.1 MB/s 
     |████████████████████████████████| 234 kB 60.5 MB/s 
     |████████████████████████████████| 6.4 MB 28.8 MB/s 
     |████████████████████████████████| 3.4 MB 54.6 MB/s 
     |████████████████████████████████| 1.1 MB 14.4 MB/s 
     |████████████████████████████████| 840 kB 51.2 MB/s 
     |████████████████████████████████| 77 kB 2.8 MB/s 
     |████████████████████████████████| 87 kB 7.4 MB/s 
     |████████████████████████████████| 120 kB 67.5 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 10.0 MB/s 
     |████████████████████████████████| 352 kB 70.8 MB/s 
     |████████████████████████████████| 47.7 MB 73 kB/s 
     |████████████████████████████████| 462 kB 72.5 MB/s 
     |██████████████

In [ ]:
#Needed imports although matplotlib, numpy,and pandas aren't currently used but will probably be needed
import tensorflow as tf

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import numpy as np
import pandas as pd
import matplotlib as plt

import os

#Get the data and base model

In [ ]:
#Picking what base model to use, efficientdet is just a starting place
#spec = model_spec.get('efficientdet_lite0')
#spec = model_spec.get('efficientdet_lite2')
spec = model_spec.get('efficientdet_lite4')

In [63]:
#Get dat data
!curl -L "https://app.roboflow.com/ds/fosD79eC34?key=xH3OhXG8fK" > data.zip
!unzip data.zip; rm data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   893  100   893    0     0   1908      0 --:--:-- --:--:-- --:--:--  1908
100 51030  100 51030    0     0  81387      0 --:--:-- --:--:-- --:--:-- 81387
Archive:  data.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: test/Flake000011_Cam0_1_2022-1-20-20-34-21-684_bmp.rf.58d00fb155c5a3f0b2883c7382c587c9.jpg  
 extracting: test/_annotations.csv   
 extracting: train/Flake000011_Cam2_1_2022-1-20-20-34-21-679_bmp.rf.4e482ea9ea773e0ef44cce224109c207.jpg  
 extracting: train/Flake000011_Cam2_1_2022-1-20-20-34-21-679_bmp.rf.93b4abf9dbe0252d35153c2db52c0ab5.jpg  
 extracting: train/Flake000011_Cam2_1_2022-1-20-20-34-21-679_bmp.rf.cad0f5c2ef762aaa71802003a5d99f8d.jpg  
 extracting: train/Flake000011_Cam3_1_2022-1-20-20-34-21

In [64]:
#I'm working on automating the jpeg and csv manipulation but right now I still hand format the csv

#!mkdir data
!mv ./test/*.jpg .
!mv ./train/*.jpg .
!mv ./valid/*.jpg .
#!mkdir annotations
#!mv ./test/*.csv ./annotations/test.csv
#!mv ./train/*.csv ./annotations/train.csv
#!mv ./valid/*.csv ./annotations/valid.csv
#!rm ./merged.csv
#!head -n 1 ./annotation/train.csv > merged.csv && tail -n+2 -q ./annotation/*.csv >> merged.csv

#Train and test

In [65]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('./data/annotations.csv')

INFO:tensorflow:Cache will be stored in /tmp/tmpspnpr0sb with prefix filename train_d155c6d48da36cf8f5f39f5ad77b0b9d. Cache_prefix is /tmp/tmpspnpr0sb/train_d155c6d48da36cf8f5f39f5ad77b0b9d
INFO:tensorflow:Cache will be stored in /tmp/tmpm3niocu1 with prefix filename val_d155c6d48da36cf8f5f39f5ad77b0b9d. Cache_prefix is /tmp/tmpm3niocu1/val_d155c6d48da36cf8f5f39f5ad77b0b9d
INFO:tensorflow:Cache will be stored in /tmp/tmpw54uo7b8 with prefix filename test_d155c6d48da36cf8f5f39f5ad77b0b9d. Cache_prefix is /tmp/tmpw54uo7b8/test_d155c6d48da36cf8f5f39f5ad77b0b9d
INFO:tensorflow:On image 0
INFO:tensorflow:On image 0
INFO:tensorflow:On image 0


In [66]:
print(test_data.label_map)

{1: 'class', 2: 'Snowflake'}


In [68]:
# train_whole_model, controls layers being trained, setting to false uses transfer learning to train and
# only trains layers that don't match model_spec.config.var_freeze_expr.
model = object_detector.create(train_data, model_spec=spec, epochs = 5, batch_size=1, train_whole_model=True, validation_data=validation_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/5
6/9 [===================>..........] - ETA: 0s - det_loss: 1.7206 - cls_loss: 1.1430 - box_loss: 0.0116 - reg_l2_loss: 0.1077 - loss: 1.8283 - learning_rate: 0.0061 - gradient_norm: 5.9389WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2706s vs `on_train_batch_end` time: 0.4071s). Check your callbacks.


9/9 [==============================] - 69s 1s/step - det_loss: 2.0075 - cls_loss: 1.1285 - box_loss: 0.0176 - reg_l2_loss: 0.1078 - loss: 2.1153 - learning_rate: 0.0047 - gradient_norm: 7.0404 - val_det_loss: 1.5222 - val_cls_loss: 1.0992 - val_box_loss: 0.0085 - val_reg_l2_loss: 0.1078 - val_loss: 1.6300
Epoch 2/5
9/9 [==============================] - 7s 811ms/step - det_loss: 1.6059 - cls_loss: 1.0656 - box_loss: 0.0108 - reg_l2_loss: 0.1078 - loss: 1.7136 - learning_rate: 8.6218e-04 - gradient_norm: 6.2724 - val_det_loss: 1.4484 - val_cls_loss: 1.0642 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.1078 - val_loss: 1.5562
Epoch 3/5
9/9 [==============================] - 5s 626ms/step - det_loss: 1.6203 - cls_loss: 1.0610 - box_loss: 0.0112 - reg_l2_loss: 0.1078 - loss: 1.7281 - learning_rate: 3.9729e-04 - gradient_norm: 4.7010 - val_det_loss: 1.4017 - val_cls_loss: 1.0397 - val_box_loss: 0.0072 - val_reg_l2_loss: 0.1078 - val_loss: 1.5095
Epoch 4/5
9/9 [==============================] 

In [69]:
model.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  multiple                  15092016  
                                                                 
 class_net/class-predict (Se  multiple                 6066      
 parableConv2D)                                                  
                                                                 
 box_net/box-predict (Separa  multiple                 10116     
 bleConv2D)                                                      
                                                                 
Total params: 15,108,198
Trainable params: 14,952,694
Non-trainable params: 155,504
_________________________________________________________________


#Test

In [70]:
#Needs a bigger test set

#Prints mAP for whole model and specifically for each piece (class)
model.evaluate(test_data, batch_size=1)

1/1 [==============================] - 4s 4s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(100, 7)
0/100
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.200
 Average Recall     (AR) @[ IoU=0.50:0.95 | ar

{'AP': 0.2,
 'AP50': 1.0,
 'AP75': 0.0,
 'AP_/Snowflake': 0.2,
 'AP_/class': -1.0,
 'APl': -1.0,
 'APm': -1.0,
 'APs': 0.2,
 'ARl': -1.0,
 'ARm': -1.0,
 'ARmax1': 0.2,
 'ARmax10': 0.2,
 'ARmax100': 0.2,
 'ARs': 0.2}

#Making and Testing the tflite version

In [71]:
# Defaults to post training full integer quantization when exported to tflite file
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmpwqfn1c63/assets


INFO:tensorflow:Assets written to: /tmp/tmpwqfn1c63/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpy84yz4jq/labelmap.txt.


INFO:tensorflow:Saving labels in /tmp/tmpy84yz4jq/labelmap.txt.


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [72]:
#Prints mAP for whole model and specifically for each piece (class)
model.evaluate_tflite('model.tflite', test_data)

1/1 [==============================] - 48s 48s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(25, 7)
0/25
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.200
 Average Recall     (AR) @[ IoU=0.50:0.95 | ar

{'AP': 0.2,
 'AP50': 1.0,
 'AP75': 0.0,
 'AP_/Snowflake': 0.2,
 'AP_/class': -1.0,
 'APl': -1.0,
 'APm': -1.0,
 'APs': 0.2,
 'ARl': -1.0,
 'ARm': -1.0,
 'ARmax1': 0.2,
 'ARmax10': 0.2,
 'ARmax100': 0.2,
 'ARs': 0.2}

#Mount google drive for exports

In [ ]:
#link drive for easy saving, although just downloading the model is easier
from google.colab import drive
drive.mount('/content/drive')